In [ ]:
# ch09 02.ipynb

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import (
    Embedding, Dense, SimpleRNN, LSTM, Bidirectional, Flatten
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from nltk.corpus import movie_reviews
from sklearn.model_selection import train_test_split

In [ ]:
# 1. 데이터 로드

# 토큰화 및 정수 인코딩

# 텍스트를 시퀀스로 변환

# 시퀀스 패딩

# 레이블 변환

# 학습테스트 데이터 분할

In [ ]:
# 2. 세가지 모델 구축, 학습
# 랜덤 시드고정(재형성)

# 모델 성능

In [ ]:
# 3. 성능 비교 및 시각화

In [ ]:
# 4. 예측 결과 분석